# Euler Riemann problem

To compute the fluxes through the interface, we will need to solve the Riemann
problem for the Euler equations.  We will formulate it such that we have
left and right primitive variable states and we want to find the unique state on the interface:

$${\bf q}_{i+1/2} = \mathcal{R}({\bf q}_{i+1/2,L}, {\bf q}_{i+1/2,R})$$

Information about the jump across this interface will be carried away from the interface by the 3 hydrodynamic waves ($u$ and $u\pm c$).  We can define 4 regions separated by the three waves, which we'll call $L$, $L_\star$, $R_\star$, and $R$.

![Riemann solution structure](riemann-waves.png)

$L$ and $R$ are just our original states&mdash;since no waves have reached them yet, the state is unchanged.  The star states are between the left and right state.  We know from the eigenvectors that all 3 primitive variables, $(\rho, u, p)$ jump across the left and right state.  But from the center eigenvector, we know only the density jumps.  That means in the star region, the only unknowns are:

$$u_\star, p_\star, \rho_{\star,L}, \rho_{\star,R}$$

## Finding the star state

To solve the Riemann problem, we need to know how much each variable changes across each of the three waves.  To complicate matters, the left and right waves can be either shocks or rarefactions.  

<div class="alert alert-info">

**Note:** We will assume a gamma-law gas for the rest of this notebook.  Keep
    in mind that many astrophysical environments need a more general gas, and
    while the expressions are different, the basic ideas will carry over to
    the general gas case.
</div>

For a gamma-law gas, we can write down analytic expressions for the change in the primitive variables across both a rarefaction and shock.  We can then solve these to find the state inbetween the left and right waves (the star state) and then compute the wave speeds.

We'll use an exact Riemann solver to find the solution on the interface.  There a lot of algebra involved in finding the expressions for the jumps across the waves and the wave speeds, which we'll skip (by see my notes).  Instead we'll just use this solver to give us the state.

One we have the interface state, we can compute the fluxes using this state.

To describe the Riemann problem, we'll create a `State` object that holds the left or right state

In [3]:
class State:
    """ a simple object to hold a primitive variable state """

    def __init__(self, p=1.0, u=0.0, rho=1.0):
        self.p = p
        self.u = u
        self.rho = rho

    def __str__(self):
        return f"rho: {self.rho}; u: {self.u}; p: {self.p}"

Now we will define a class that find the star state.  This 

In [4]:
class RiemannProblem:
    """ a class to define a Riemann problem.  It takes a left
        and right state.  Note: we assume a constant gamma """

    def __init__(self, left_state, right_state, gamma=1.4):
        self.left = left_state
        self.right = right_state
        self.gamma = gamma

        self.ustar = None
        self.pstar = None

    def u_hugoniot(self, p, side):
        """define the Hugoniot curve, u(p)."""

        if side == "left":
            state = self.left
            s = 1.0
        elif side == "right":
            state = self.right
            s = -1.0

        c = np.sqrt(self.gamma*state.p/state.rho)

        if p < state.p:
            # rarefaction
            u = state.u + s*(2.0*c/(self.gamma-1.0))* \
                (1.0 - (p/state.p)**((self.gamma-1.0)/(2.0*self.gamma)))
        else:
            # shock
            beta = (self.gamma+1.0)/(self.gamma-1.0)
            u = state.u + s*(2.0*c/np.sqrt(2.0*self.gamma*(self.gamma-1.0)))* \
                (1.0 - p/state.p)/np.sqrt(1.0 + beta*p/state.p)

        return u

    def find_star_state(self, p_min=0.001, p_max=1000.0):
        """ root find the Hugoniot curve to find ustar, pstar """

        # we need to root-find on
        self.pstar = optimize.brentq(
            lambda p: self.u_hugoniot(p, "left") - self.u_hugoniot(p, "right"),
            p_min, p_max)
        self.ustar = self.u_hugoniot(self.pstar, "left")

## Sampling the solution

Finally, we can find the solution on the interface by determining which region we are in.
![Riemann state](riemann_state.png)